## 如何开始训练？

### pytorch 补充

transformer的库相对来说封装完整，我们不必过度依赖pytorch进行训练过程，但数据的加载、模型参数调优等等还需要通过pytorch实现。使用 Pytorch 进行计算的好处是更高效的执行速度，尤其当张量存储的数据很多时，而且还可以借助 GPU 进一步提高计算速度。

张量 (Tensor) 是深度学习的基础，例如常见的 0 维张量称为标量 (scalar)、1 维张量称为向量 (vector)、2 维张量称为矩阵 (matrix)。Pytorch 本质上就是一个基于张量的数学计算工具包，它提供了多种方式来创建张量：

针对张量的计算：

In [5]:
import torch
import numpy as np
a=torch.empty(2, 3) # empty tensor (uninitialized), shape (2,3)
print('----------a----------')
print(a)
b=torch.rand(2, 3) # random tensor, each value taken from [0,1)
print('----------b----------')
print(b)
print('----------c----------')
c= torch.randn(2, 3) # random tensor, each value taken from standard normal distribution
print(c)
print('----------d----------')
d= torch.zeros(2, 3, dtype=torch.long) # long integer zero tensor
print(d)
print('----------e----------')
e= torch.zeros(2, 3, dtype=torch.double) # double float zero tensor
print(e)
print('----------f----------')
f= torch.arange(10)
print(f)
print('-------array1---------')
## 也可以直接基于numpy转化
array = [[1.0, 3.8, 2.1], [8.6, 4.0, 2.4]]
print(torch.tensor(array))
print('-------array2---------')
array = np.array([[1.0, 3.8, 2.1], [8.6, 4.0, 2.4]])
print(torch.from_numpy(array))


----------a----------
tensor([[0., 0., 0.],
        [0., 0., 0.]])
----------b----------
tensor([[0.2977, 0.0095, 0.3499],
        [0.2987, 0.2878, 0.1401]])
----------c----------
tensor([[ 0.5798, -0.0051, -1.5515],
        [-0.3403, -0.5510, -0.7097]])
----------d----------
tensor([[0, 0, 0],
        [0, 0, 0]])
----------e----------
tensor([[0., 0., 0.],
        [0., 0., 0.]], dtype=torch.float64)
----------f----------
tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [17]:
x = torch.tensor([1, 2, 3], dtype=torch.double)
y = torch.tensor([4, 5, 6], dtype=torch.double)
# 加减乘除
print(x + y)
print(x - y)
print(x * y)
print(x / y)
print(x.dot(y))
a=torch.cat((x, y),dim=0)
print(a)


tensor([5., 7., 9.], dtype=torch.float64)
tensor([-3., -3., -3.], dtype=torch.float64)
tensor([ 4., 10., 18.], dtype=torch.float64)
tensor([0.2500, 0.4000, 0.5000], dtype=torch.float64)
tensor(32., dtype=torch.float64)
tensor([1., 2., 3., 4., 5., 6.], dtype=torch.float64)


### pytorch支持梯度的自动计算。
可以自动计算一个函数关于一个变量在某一取值下的导数，从而基于梯度对参数进行优化，这就是机器学习中的训练过程。使用 Pytorch 计算梯度非常容易，只需要执行 tensor.backward()，就会自动通过反向传播 (Back Propogation) 算法完成，后面我们在训练模型时就会用到该函数。同时，还可以通过view，reshape，permute，transpose等函数实现张量尺寸的变化重构。以及实现索引、切片、等功能。 请参考https://pytorch.org/docs/stable/index.html，以及课下资料进行补充学习。

In [19]:
x = torch.tensor([2.], requires_grad=True)
y = torch.tensor([3.], requires_grad=True)
z = (x + y) * (y - 2)
print(z)
z.backward()
print(x.grad, y.grad)

tensor([5.], grad_fn=<MulBackward0>)
tensor([1.]) tensor([6.])


### 数据处理（dataloaders）
在实际训练模型时，我们都需要先将数据集切分为很多的 mini-batches，然后按批 (batch) 将样本送入模型，并且循环这一过程，每一个完整遍历所有样本的循环称为一个 epoch。

Pytorch 提供了 DataLoader 类专门负责处理这些操作，除了基本的 dataset（数据集）和 batch_size （batch 大小）参数以外，还有以下常用参数：

shuffle：是否打乱数据集；
sampler：采样器，也就是一个索引上的迭代器；
collate_fn：批处理函数，用于对采样出的一个 batch 中的样本进行处理（比如当最后一个batch数量不足，进行填0）。

当需要对数据进行采样时，还需要定义sampler类，作为采样器。比如我们可以乱序地为每一轮数据训练读入数据。

### 数据加载实例：

- 导入了必要的库和模块。

- 加载了FashionMNIST数据集的训练集和测试集，并将图片转换为PyTorch张量。

- 创建了训练集和测试集的DataLoader，用于在训练和测试时批量获取数据。

- 从DataLoader中获取了第一批数据的特征和标签，并打印了它们的形状。

- 取出了特征批次中的第一张图片和对应的标签，并打印了图片的形状和标签的值。


In [21]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# 从torchvision库中导入FashionMNIST数据集
training_data = datasets.FashionMNIST(
    root="data",  # 指定数据存储的目录
    train=True,  # 指定加载训练集
    download=True,  # 如果数据集不存在，则下载数据集
    transform=ToTensor()  # 将图片转换为PyTorch张量
)

# 从torchvision库中导入FashionMNIST数据集，用于测试集
test_data = datasets.FashionMNIST(
    root="data",  # 指定数据存储的目录
    train=False,  # 指定加载测试集
    download=True,  # 如果数据集不存在，则下载数据集
    transform=ToTensor()  # 将图片转换为PyTorch张量
)

# 创建训练数据的DataLoader，设置批处理大小为64，并且数据在每个epoch开始时随机打乱
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
# 创建测试数据的DataLoader，设置批处理大小为64，并且数据在每个epoch开始时随机打乱
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

# 使用DataLoader的迭代器获取第一批数据的特征和标签
train_features, train_labels = next(iter(train_dataloader))
# 打印特征批次的形状，即每批有多少张图片，每张图片的尺寸是多少
print(f"Feature batch shape: {train_features.size()}")
# 打印标签批次的形状，即每批有多少个标签
print(f"Labels batch shape: {train_labels.size()}")

# 从特征批次中取出第一张图片，并使用squeeze()方法去除维度为1的轴，使其成为二维数组
img = train_features[0].squeeze()
# 从标签批次中取出第一个标签
label = train_labels[0]
# 打印图片的形状，即图片的尺寸
print(img.shape)
# 打印标签的值
print(f"Label: {label}")


100%|███████████████████████████| 26421880/26421880 [00:33<00:00, 787865.22it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|█████████████████████████████████| 29515/29515 [00:00<00:00, 185793.67it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████████████████████████| 4422102/4422102 [01:22<00:00, 53778.94it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████████████████████████████| 5148/5148 [00:00<00:00, 3987493.44it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Feature batch shape: torch.Size([64, 1, 28, 28])
Labels batch shape: torch.Size([64])
torch.Size([28, 28])
Label: 3


### 下面我们使用PyTorch框架实现了简单的神经网络训练和测试流程，主要用于图像分类任务。代码的主要结构和功能如下：

### 环境设置：

检查是否有可用的CUDA设备（GPU），如果有则使用GPU进行计算，否则使用CPU。

### 数据加载：

使用datasets.FashionMNIST加载FashionMNIST数据集，这是一个包含10个类别的服装图像数据集。 数据集被分为训练集和测试集，并且通过ToTensor()转换为PyTorch张量。 使用DataLoader创建数据加载器，以便在训练和测试时批量获取数据。

### 模型定义：

定义一个名为NeuralNetwork的类，它继承自nn.Module，表示一个简单的前馈神经网络。 网络包含一个展平层（nn.Flatten），将28x28的图像展平为784维的向量。 接着是三个全连接层（nn.Linear），中间两个全连接层后跟ReLU激活函数，最后一个全连接层后跟Dropout层以减少过拟合。

### 训练和测试函数：

train_loop：定义训练循环，负责模型的训练过程，包括前向传播、计算损失、反向传播和参数更新。 test_loop：定义测试循环，负责模型的评估过程，计算测试集上的损失和准确率。

### 优化器和损失函数：

使用nn.CrossEntropyLoss作为损失函数，适用于多分类问题。 使用torch.optim.AdamW作为优化器，这是一种带有权重衰减的Adam优化算法。

### 训练过程：

通过循环epochs次，每次循环中调用train_loop进行训练，调用test_loop进行测试，并打印出每个周期的损失和准确率。 输出：

在训练结束后，打印"Done!"表示训练完成。

In [22]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# 检查是否有可用的CUDA设备，如果有则使用GPU，否则使用CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

# 加载FashionMNIST训练数据集，并将图片转换为Tensor
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

# 加载FashionMNIST测试数据集，并将图片转换为Tensor
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

# 设置学习率、批处理大小和训练周期
learning_rate = 1e-3
batch_size = 64
epochs = 3

# 创建训练数据的DataLoader
train_dataloader = DataLoader(training_data, batch_size=batch_size)
# 创建测试数据的DataLoader
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# 定义神经网络结构
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # 将图片展平为一维向量
        self.flatten = nn.Flatten()
        # 定义一个包含两个隐藏层的前馈神经网络
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 10),
            nn.Dropout(p=0.2)  # 在最后一个全连接层后添加Dropout防止过拟合
        )

    def forward(self, x):
        # 前向传播过程
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# 实例化模型并将其移动到GPU或CPU
model = NeuralNetwork().to(device)

# 定义训练循环函数
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()  # 设置模型为训练模式
    for batch, (X, y) in enumerate(dataloader, start=1):
        X, y = X.to(device), y.to(device)  # 将数据移动到GPU或CPU
        # 计算预测结果和损失
        pred = model(X)
        loss = loss_fn(pred, y)
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# 定义测试循环函数
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    model.eval()  # 设置模型为评估模式
    with torch.no_grad():  # 在评估模式下不计算梯度
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(dim=-1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# 定义损失函数和优化器
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# 训练和测试模型
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Using cpu device
Epoch 1
-------------------------------
loss: 0.983582  [ 6400/60000]
loss: 0.872947  [12800/60000]
loss: 0.538679  [19200/60000]
loss: 0.889003  [25600/60000]
loss: 0.480309  [32000/60000]
loss: 0.538123  [38400/60000]
loss: 0.856580  [44800/60000]
loss: 0.611855  [51200/60000]
loss: 0.475762  [57600/60000]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.464763 

Epoch 2
-------------------------------
loss: 0.606370  [ 6400/60000]
loss: 0.521568  [12800/60000]
loss: 0.557305  [19200/60000]
loss: 0.790215  [25600/60000]
loss: 0.476856  [32000/60000]
loss: 0.451210  [38400/60000]
loss: 0.938261  [44800/60000]
loss: 0.653995  [51200/60000]
loss: 0.495939  [57600/60000]
Test Error: 
 Accuracy: 84.9%, Avg loss: 0.418934 

Epoch 3
-------------------------------
loss: 0.579286  [ 6400/60000]
loss: 0.642542  [12800/60000]
loss: 0.432076  [19200/60000]
loss: 0.712900  [25600/60000]
loss: 0.340836  [32000/60000]
loss: 0.378215  [38400/60000]
loss: 0.820740  [44800/60000]
loss: 0.4